## Vehicle Routing Problem

working on input : design layout warehouse 2D

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name

role = get_execution_role()

print(role)

arn:aws:iam::924756873743:role/TeamRole


In [8]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-opt-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

print(processing_repository_uri)

924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container:latest


In [11]:
!docker build -t $ecr_repository docker

Sending build context to Docker daemon  2.048kB
Step 1/5 : FROM continuumio/anaconda3
latest: Pulling from continuumio/anaconda3

50cd189d: Pulling fs layer 
1e8957d3: Pulling fs layer 
Digest: sha256:0b2047cdc438807b87d53272c3d5b10c8238fe65a2fedf9bd72de0b7ba360cb1[1A
Status: Downloaded newer image for continuumio/anaconda3:latest
 ---> 5e5dd010ead8
Step 2/5 : RUN pip install boto3 pandas scikit-learn pulp pyomo inspyred ortools scipy deap
 ---> Running in b129229c7484
  Created wheel for boto3: filename=boto3-1.17.49-py2.py3-none-any.whl size=128780 sha256=df0b8ed4f4428590c5801274676b82414d044d2ada2e668299d86b24912583c9
  Stored in directory: /root/.cache/pip/wheels/3d/9d/e0/c1928c18746d5c0a4e1d581da31392611493c89d3f45390d50
Successfully built boto3
Removing intermediate container b129229c7484
 ---> 79b6e4c25a58
Step 3/5 : RUN conda install -c conda-forge ipopt coincbc glpk
 ---> Running in a4e261450439
Solving environment: ...working... failed with initial frozen solve. Retrying with

python_abi-3.8       | 4 KB      | ########## | 100% 
libblas-3.8.0        | 11 KB     | ########## | 100% 
libcblas-3.8.0       | 11 KB     | ########## | 100% 
coincbc-2.10.5       | 7.9 MB    | ########## | 100% 
conda-4.10.0         | 3.1 MB    | ########## | 100% 
glpk-4.65            | 1.2 MB    | ########## | 100% 
liblapack-3.8.0      | 11 KB     | ########## | 100% 
metis-5.1.0          | 4.1 MB    | ########## | 100% 
ipopt-3.13.2         | 2.0 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base -c defaults conda


Removing intermediate container a4e261450439
 ---> 51d54cb350ce
Step 4/5 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in a0cb4bd5c4d6
Removing intermediate container a0cb4bd5c4d6
 ---> 3eeda17ea68b
Step 5/5 : 

In [12]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:924756873743:repository/sagemaker-opt-container",
        "registryId": "924756873743",
        "repositoryName": "sagemaker-opt-container",
        "repositoryUri": "924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container",
        "createdAt": 1618029456.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
The push refers to repository [924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container]

88055bb4: Preparing 
f

d44a0a55: Pushing  1.661GB/2.427GBd44a0a55: Pushed   2.481GB/2.427GBlatest: digest: sha256:ae59ab5db9cff34e878b441cbb369a60eae14a097767879e0f7670b2e0423143 size: 1380


In [19]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(command=['python'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.c4.8xlarge')

In [31]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-17-03-31-475
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-03-31-475/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-03-31-475/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
........................../opt/conda/lib/python3.8/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /opt/conda/lib/

In [32]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-17-33-02-297
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-33-02-297/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-33-02-297/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
...........................
/opt/conda/lib/python3.8/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /opt/conda/li

{'ProcessingInputs': [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-33-02-297/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-33-02-297/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}]}, 'ProcessingJobName': 'sagemaker-opt-container-2021-04-10-17-33-02-297', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.c4.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container:

In [33]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-17-39-48-543
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-39-48-543/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-39-48-543/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
..........................0-0 SUPPLY
2-2 DEMAND
2-3 DEMAND
2-6 DEMAND
2-7 DEMAND
2-10 DEMAND
2-11 DEMAND
2-14 DEMAND
2-15 DEMAND
5-2 DEMAND
5-3 DEMAND
5-6 DEMAND
5-7 DEMAND
5-10 DEMAND
5-11 DEMAND
5-14 DEMAND
5-15 DEMAND
6-2 DEMAND
6-3 DEMAND
6-6 DEMAND
6-7 DEMAND
6-10 DEMAND
6-11 DEMAND
6-14 DEMAND
6-15 DEMAND
9-2 DEMAND
9-3 DE


{'ProcessingInputs': [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-39-48-543/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-39-48-543/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}]}, 'ProcessingJobName': 'sagemaker-opt-container-2021-04-10-17-39-48-543', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.c4.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container

In [34]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-17-55-20-945
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-55-20-945/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-17-55-20-945/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
............................0-0 SUPPLY
2-2 DEMAND
2-3 DEMAND
2-6 DEMAND
2-7 DEMAND
2-10 DEMAND
2-11 DEMAND
2-14 DEMAND
2-15 DEMAND
5-2 DEMAND
5-3 DEMAND
5-6 DEMAND
5-7 DEMAND
5-10 DEMAND
5-11 DEMAND
5-14 DEMAND
5-15 DEMAND
6-2 DEMAND
6-3 DEMAND
6-6 DEMAND
6-7 DEMAND
6-10 DEMAND
6-11 DEMAND
6-14 DEMAND
6-15 DEMAND
9-2 DEMAND
9-3 

In [35]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-18-03-25-689
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-03-25-689/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-03-25-689/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
...........................
SUPPLY 0-0 100 0
DEMAND 2-2 0 6
DEMAND 2-3 0 7
DEMAND 2-6 0 10
DEMAND 2-7 0 11
DEMAND 2-10 0 14
DEMAND 2-11 0 15
DEMAND 2-14 0 18
DEMAND 2-15 0 19
DEMAND 5-2 0 9
DEMAND 5-3 0 10
DEMAND 5-6 0 13
DEMAND 5-7 0 14
DEMAND 5-10 0 17
DEMAND 5-11 0 18
DEMAND 5-14 0 21
DEMAND 5-15 0 22
DEMAND 6-2 0 10
DEMAND 6

In [38]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-18-47-54-575
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-47-54-575/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-47-54-575/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
..........................SUPPLY 0-0 100 0
DEMAND 2-2 0 6
DEMAND 2-3 0 7
DEMAND 2-6 0 10
DEMAND 2-7 0 11
DEMAND 2-10 0 14
DEMAND 2-11 0 15
DEMAND 2-14 0 18
DEMAND 2-15 0 19
DEMAND 5-2 0 9
DEMAND 5-3 0 10
DEMAND 5-6 0 13
DEMAND 5-7 0 14
DEMAND 5-10 0 17
DEMAND 5-11 0 18
DEMAND 5-14 0 21
DEMAND 5-15 0 22
/opt/conda/lib/python3.8/s


{'ProcessingInputs': [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-47-54-575/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'data', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-47-54-575/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}]}, 'ProcessingJobName': 'sagemaker-opt-container-2021-04-10-18-47-54-575', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1, 'InstanceType': 'ml.c4.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '924756873743.dkr.ecr.us-east-1.amazonaws.com/sagemaker-opt-container

In [39]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-18-55-24-922
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-55-24-922/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-18-55-24-922/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
............................SUPPLY 0-0 100 0
DEMAND 2-2 0 6
DEMAND 2-3 0 7
DEMAND 2-6 0 10
DEMAND 2-7 0 11
DEMAND 2-10 0 14
DEMAND 2-11 0 15
DEMAND 2-14 0 18
DEMAND 2-15 0 19
DEMAND 5-2 0 9
DEMAND 5-3 0 10
DEMAND 5-6 0 13
DEMAND 5-7 0 14
DEMAND 5-10 0 17
DEMAND 5-11 0 18
DEMAND 5-14 0 21
DEMAND 5-15 0 22
/opt/conda/lib/python3.8

In [40]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-19-02-42-470
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-19-02-42-470/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-19-02-42-470/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
........................SUPPLY 0-0 500 0
SUPPLY 1-0 500 0
DEMAND 2-2 0 6
DEMAND 2-3 0 7
DEMAND 2-6 0 10
DEMAND 2-7 0 11
DEMAND 2-10 0 14
DEMAND 2-11 0 15
DEMAND 2-14 0 18
DEMAND 2-15 0 19
DEMAND 5-2 0 9
DEMAND 5-3 0 10
DEMAND 5-6 0 13
DEMAND 5-7 0 14
DEMAND 5-10 0 17
DEMAND 5-11 0 18
DEMAND 5-14 0 21
DEMAND 5-15 0 22
DEMAND 6-2 

In [41]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocessing.py',
                      outputs=[ProcessingOutput(output_name='data',
                                                source='/opt/ml/processing/data')])

script_processor_job_description = script_processor.jobs[-1].describe()

print(script_processor_job_description)


Job Name:  sagemaker-opt-container-2021-04-10-19-16-03-678
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-19-16-03-678/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-924756873743/sagemaker-opt-container-2021-04-10-19-16-03-678/output/data', 'LocalPath': '/opt/ml/processing/data', 'S3UploadMode': 'EndOfJob'}}]
...........................
SUPPLY 0-0 500 0
DEMAND 2-2 0 6
DEMAND 2-3 0 7
DEMAND 2-6 0 10
DEMAND 2-7 0 11
DEMAND 2-10 0 14
DEMAND 2-11 0 15
DEMAND 2-14 0 18
DEMAND 2-15 0 19
DEMAND 5-2 0 9
DEMAND 5-3 0 10
DEMAND 5-6 0 13
DEMAND 5-7 0 14
DEMAND 5-10 0 17
DEMAND 5-11 0 18
DEMAND 5-14 0 21
DEMAND 5-15 0 22
DEMAND 6-2 0 10
DEMAND 6